In [ ]:
import torch

In [ ]:
N = 20
M = 10

In [ ]:
W1 = torch.randn([N,M])
W2 = W1 + torch.randn([N,M]) / 10

In [ ]:
A_pre = torch.nn.Parameter(torch.eye(N))
A_post = torch.nn.Parameter(torch.eye(M))

### A_pre

In [ ]:
optimiser = torch.optim.Adam([A_pre], lr = 1e-3)
criteria = torch.nn.MSELoss()

In [ ]:
for i in range(10000):
    optimiser.zero_grad()
    loss = criteria(W2, A_pre @ W1)
    loss.backward()
    optimiser.step()
    if i%1000==0:
        print(loss)

### A_post

In [ ]:
optimiser = torch.optim.Adam([A_post], lr = 1e-3)
criteria = torch.nn.MSELoss()

In [ ]:
for i in range(10000):
    optimiser.zero_grad()
    loss = criteria(W2, W1 @ A_post)
    loss.backward()
    optimiser.step()
    if i%1000==0:
        print(loss)

In [ ]:
def init_lora_matrices(A, B):
    N,R = A.shape
    A.requires_grad = False
    B.requires_grad = False
    B[0][0] = 1
    A[:,0] = 1
    A.requires_grad = True
    B.requires_grad = True

In [ ]:
shift_indices = None

def set_shift_indices(r, c):
    global shift_indices
    shift_indices = torch.zeros([r,c], dtype=torch.int64)
    for i in range(r):
        for j in range(c):
            shift_indices[i][j] = (j - i)%c

def shift(M):
    n_rows, n_cols = M.shape
    global shift_indices
    return torch.gather(M, 1, shift_indices)

### A_post_r

In [ ]:
R = 1
while R <= N:
    print("#"*5 + " Rank Post " + str(R) + " " + "#"*5)
    A1 = torch.nn.Parameter(torch.randn([N,R]))
    B1 = torch.nn.Parameter(torch.zeros([R,N]))
    init_lora_matrices(A1,B1)
    optimiser = torch.optim.Adam([A1, B1], lr = 1e-3)
    criteria = torch.nn.MSELoss()
    for i in range(10000):
        optimiser.zero_grad()
        loss = criteria(W2, shift(A1 @ B1) @ W1)
        loss.backward()
        optimiser.step()
    print(loss)
    R=R*2

### A_pre_r

In [ ]:
R = 1
while R <= M:
    print("#"*5 + " Rank Pre " + str(R) + " " + "#"*5)
    A1 = torch.nn.Parameter(torch.randn([M,R]))
    B1 = torch.nn.Parameter(torch.zeros([R,M]))
    init_lora_matrices(A1,B1)
    optimiser = torch.optim.Adam([A1, B1], lr = 1e-3)
    criteria = torch.nn.MSELoss()
    for i in range(10000):
        optimiser.zero_grad()
        loss = criteria(W2, W1 @ shift(A1 @ B1))
        loss.backward()
        optimiser.step()
    print(loss)
    R=R*2

### LoRA

In [ ]:
R = 1
while R <= M:
    print("#"*5 + " LoRA " + str(R) + " " + "#"*5)
    A1 = torch.nn.Parameter(torch.randn([N,R]))
    B1 = torch.nn.Parameter(torch.zeros([R,M]))
    optimiser = torch.optim.Adam([A1, B1], lr = 1e-3)
    criteria = torch.nn.MSELoss()
    for i in range(10000):
        optimiser.zero_grad()
        loss = criteria(W2, W1 + A1 @ B1)
        loss.backward()
        optimiser.step()
    print(loss)
    R=R*2

### SLoRA

In [ ]:
R = 1
while R <= M:
    print("#"*5 + " SLoRA " + str(R) + " " + "#"*5)
    A1 = torch.nn.Parameter(torch.randn([N,R]))
    B1 = torch.nn.Parameter(torch.zeros([R,M]))
    set_shift_indices(N,M)
    optimiser = torch.optim.Adam([A1, B1], lr = 1e-3)
    criteria = torch.nn.MSELoss()
    for i in range(10000):
        optimiser.zero_grad()
        loss = criteria(W2, W1 + shift(A1 @ B1))
        loss.backward()
        optimiser.step()
    print(loss)
    R=R*2

In [ ]:
W1 = torch.randn([N,M])
W2 = W1 + torch.randn([N,M]) / 10

In [ ]:
N = 128
M = 512

In [ ]:
torch.set_default_device('cuda')